In [15]:
import pandas as pd
import yaml
import torch
from pytorch_lightning import Trainer, seed_everything
from eyemind.trainer.loops import KFoldLoop
from eyemind.models.classifier import EncoderClassifierMultiSequenceModel, EncoderClassifierModel
from eyemind.models.encoder_decoder import VariableSequenceLengthEncoderDecoderModel
from eyemind.dataloading.gaze_data import SequenceToLabelDataModule, SequenceToSequenceDataModule

In [2]:
config_path = "/Users/rickgentry/emotive_lab/eyemind/experiment_configs/local/comprehension_nestedcv_test_config.yml"
with open(config_path, "r") as f:
    config = yaml.safe_load(f)

In [3]:
seed_everything(config["seed_everything"], workers=True)

Global seed set to 42


42

## Load Model checkpoint

In [4]:
ckpt_path = "/Users/rickgentry/emotive_lab/eyemind/ray_results/comprehension-tune-sub/fold3_freeze_encoder=False/checkpoints/epoch=19-step=460.ckpt"    
model = EncoderClassifierModel.load_from_checkpoint(ckpt_path,encoder_weights_path=None)

CNN Layers: [16, 32]
16 2
32 16


/Users/rickgentry/miniforge3/envs/eyemind/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


## Setup Data and Trainer

In [5]:
datamodule = SequenceToLabelDataModule(**config["data"])
datamodule.setup()

In [6]:
datamodule.setup_cv_folds(4,4)

In [7]:
datamodule.setup_cv_fold_index(3,-1)

In [8]:
trainer = Trainer(**config["trainer"])

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


## Predictions

In [9]:
train_dl = datamodule.get_dataloader(datamodule.train_fold)
val_dl = datamodule.get_dataloader(datamodule.val_fold)
test_dl = datamodule.test_dataloader()

In [12]:
train_preds = trainer.predict(model, train_dl)
val_preds = trainer.predict(model, val_dl)
test_preds = trainer.predict(model, test_dl)

/Users/rickgentry/miniforge3/envs/eyemind/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Predicting DataLoader 0: 100%|██████████| 12/12 [00:11<00:00,  1.06it/s]


In [16]:
def get_fraction_of_comprehension(preds):
    return torch.cat(preds).mean()

In [19]:
print(f"train:{get_fraction_of_comprehension(train_preds)}, val: {get_fraction_of_comprehension(val_preds)}, test: {get_fraction_of_comprehension(test_preds)}")

train:0.6068152189254761, val: 0.580152690410614, test: 0.591160237789154


## Predictive Coding

In [1]:
from eyemind.obf.utils.signal_visualization import *
from eyemind.analysis.predictions import *
from eyemind.models.encoder_decoder import *


/Users/rickgentry/miniforge3/envs/eyemind/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
checkpoint_path = "/Users/rickgentry/emotive_lab/eyemind/lightning_logs/version_11471837/checkpoints/epoch=99-step=1600.ckpt"

In [3]:
model = load_model_from_checkpoint(MultiTaskEncoderDecoder, checkpoint_path)

CNN Layers: [16, 32]
16 2
32 16
CNN Layers: [16, 32]
16 2
32 16


/Users/rickgentry/miniforge3/envs/eyemind/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


In [4]:
dl = get_dataloader("/Users/rickgentry/emotive_lab/eyemind/lightning_logs/version_11471837/config.yaml")

In [5]:
for batch in dl:
    preds = model.predict_step(batch, 0)
    break


ValueError: not enough values to unpack (expected 4, got 2)